[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/finance-data-analysis/blob/main/3.3%20%EB%84%A4%EC%9D%B4%EB%B2%84%EA%B8%88%EC%9C%B5%20%EA%B0%9C%EB%B3%84%EC%A2%85%EB%AA%A9%20%EC%88%98%EC%A7%91-input.ipynb)


## 네이버 금융 개별종목 수집
* FinanceDataReader를 통해 수집했던 데이터를 네이버 증권 웹 페이지를 통해 직접 수집합니다.


### Keyword

* html 파일 읽어오기
    * pd.read_html(url, encoding="cp949")

* 결측 데이터 제거하기(axis 0:행, 1:열)
    * table[0].dropna()

* 데이터 프레임 합치기
    * pd.concat([df1, df2, df3])

* 중복데이터 제거
    * df.drop_duplicates()

* 과학적 기수법
    * 1.210000e+02 => 121

* 날짜 column의 첫 row값 확인
    * date = df.iloc[0]["날짜"]

* 파일로 저장하기 
    * df.to_csv(file_name, index=False)

* 파일 읽어오기
    * pd.read_csv(file_name)

### 주요 라이브러리 소개
__1. requests__
- 작은 브라우저로 웹사이트를 읽어오는 목적


__2. beautifulSoup__
- 읽어온 웹사이트의 HTML 문서를 해석하는 목적으로 사용


__3. tqdm__
- 반복되는 오래걸리는 작업을 할 때 사용

## 수집할 페이지 보기

* 네이버 금융 국내증시 : https://finance.naver.com/sise/
* 2020년 주요 상장종목
    * 빅히트 : https://finance.naver.com/item/main.nhn?code=352820
    * 카카오게임즈 : https://finance.naver.com/item/main.nhn?code=293490
    * SK바이오팜 : https://finance.naver.com/item/main.nhn?code=326030

## 라이브러리 로드

In [1]:
# 라이브러리 로드
import pandas as pd

## 수집할 URL 정하기

In [26]:
# 종목번호와 상장사 이름을 item_code와 item_name으로 설정
# item_code = "352820"
# item_name = "빅히트"

item_code = "326030"
item_name = "SK바이오팜"

# 종목 URL 만들기
url = 'https://finance.naver.com/item/sise_day.naver?code=005930&page=1'

## requests를 통한 HTTP 요청
* [Requests: HTTP for Humans™ — Requests documentation](https://requests.readthedocs.io/en/master/)
* [Quickstart — Requests documentation # custom-headers](https://requests.readthedocs.io/en/latest/user/quickstart/#custom-headers)

In [31]:
import requests

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}

response = requests.get(url, headers=headers)
response

<Response [200]>

In [32]:
response.text

'\n<html lang="ko">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr">\n<title>네이버 금융</title>\n\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/newstock.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/common.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/layout.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/main.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/newstock2.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/newstock3.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/world.css">\n</head>\n<body>\n<script

## BeautifulSoup 을 통한 table 태그 찾기

* [Beautiful Soup Documentation — Beautiful Soup 4.9.0 documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [35]:
from bs4 import BeautifulSoup as bs

html = bs(response.text)
html

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>네이버 금융</title>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/newstock.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/common.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/layout.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/main.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/newstock2.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/newstock3.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/world.css" rel="stylesheet" type="text/css"/>
</head>
<body>
<script language="

In [40]:
# table 태그만 가져오기
table = html.select("table")
len(table)

2

In [42]:
table

[<table cellspacing="0" class="type2">
 <tr>
 <th>날짜</th>
 <th>종가</th>
 <th>전일비</th>
 <th>시가</th>
 <th>고가</th>
 <th>저가</th>
 <th>거래량</th>
 </tr>
 <tr>
 <td colspan="7" height="8"></td>
 </tr>
 <tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
 <td align="center"><span class="tah p10 gray03">2021.10.14</span></td>
 <td class="num"><span class="tah p11">69,400</span></td>
 <td class="num">
 <img alt="상승" height="6" src="https://ssl.pstatic.net/imgstock/images/images4/ico_up.gif" style="margin-right:4px;" width="7"/><span class="tah p11 red02">
 				600
 				</span>
 </td>
 <td class="num"><span class="tah p11">69,000</span></td>
 <td class="num"><span class="tah p11">69,800</span></td>
 <td class="num"><span class="tah p11">68,800</span></td>
 <td class="num"><span class="tah p11">19,337,498</span></td>
 </tr>
 <tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
 <td align="center"><span class="tah p10 gray03">2021.10.13</span></td>
 <td class="num"><span class="t

## pandas 코드 한 줄로 데이터 수집하기

In [46]:
# read_html을 이용하여 url의 page내의 값을 DataFrame으로 받아옵니다.
# cp949는 한글 인코딩을 위해 사용합니다. 기본 인코딩 설정은 utf-8 이며, 
# 네이버의 일별 시세는 cp949 인코딩으로 불러올 수 있습니다.
# 데이터를 로드 했을 때 한글 인코딩이 깨진다면 대부분 cp949 로 불러올 수 있습니다.
# table
pd.read_html(str(table))[0]

,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021.10.14,69400.0,600.0,69000.0,69800.0,68800.0,19337498.0
2,2021.10.13,68800.0,200.0,68700.0,69600.0,68300.0,24172015.0
3,2021.10.12,69000.0,2500.0,70700.0,70900.0,68700.0,31001484.0
4,2021.10.08,71500.0,100.0,72300.0,72400.0,71500.0,14043287.0
5,2021.10.07,71600.0,300.0,71600.0,72100.0,71300.0,13683532.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021.10.06,71300.0,900.0,72600.0,72800.0,71200.0,18956962.0


In [9]:
# table[0]와 table[1]을 확인하여 보면 table[0]에 필요한 데이터들이 있습니다.
table[0]

,0,1,2
0,"전일 68,800 68,800","고가 69,80069,800 (상한가 89,40089,400 )","거래량 19,337,498 19,337,498"
1,"시가 69,00069,000","저가 68,80068,800 (하한가 48,200 )","거래대금 1,339,832 1,339,832 백만"


In [49]:
# dropna를 통해 결측치가 들어있는 row를 제거합니다.
# temp
temp = pd.read_html(str(table))[0]
temp = temp.dropna()
temp

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.10.14,69400.0,600.0,69000.0,69800.0,68800.0,19337498.0
2,2021.10.13,68800.0,200.0,68700.0,69600.0,68300.0,24172015.0
3,2021.10.12,69000.0,2500.0,70700.0,70900.0,68700.0,31001484.0
4,2021.10.08,71500.0,100.0,72300.0,72400.0,71500.0,14043287.0
5,2021.10.07,71600.0,300.0,71600.0,72100.0,71300.0,13683532.0
9,2021.10.06,71300.0,900.0,72600.0,72800.0,71200.0,18956962.0
10,2021.10.05,72200.0,1000.0,73000.0,73000.0,71400.0,24013921.0
11,2021.10.01,73200.0,900.0,73900.0,74000.0,72900.0,15803395.0
12,2021.09.30,74100.0,0.0,74300.0,74800.0,73700.0,19919361.0
13,2021.09.29,74100.0,2200.0,74800.0,75300.0,73800.0,23992458.0


## 페이지별 데이터 수집 함수 만들기

In [50]:
item_code = "089860"
item_name = "롯데렌탈"
page_no = 1

# 종목 URL 만들기
url = f"https://finance.naver.com/item/sise_day.naver?code={item_code}&page={page_no}"
url

'https://finance.naver.com/item/sise_day.naver?code=089860&page=1'

In [51]:
# requests url
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}

response = requests.get(url, headers=headers)
response

<Response [200]>

In [52]:
response.text

'\n<html lang="ko">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr">\n<title>네이버 금융</title>\n\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/newstock.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/common.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/layout.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/main.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/newstock2.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/newstock3.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/world.css">\n</head>\n<body>\n<script

In [53]:
# BeautifulSoup 
html = bs(response.text)
html

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>네이버 금융</title>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/newstock.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/common.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/layout.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/main.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/newstock2.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/newstock3.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20211013101915/css/world.css" rel="stylesheet" type="text/css"/>
</head>
<body>
<script language="

In [54]:
# table 태그만 가져오기
table = html.select("table")
len(table)

2

In [55]:
table

[<table cellspacing="0" class="type2">
 <tr>
 <th>날짜</th>
 <th>종가</th>
 <th>전일비</th>
 <th>시가</th>
 <th>고가</th>
 <th>저가</th>
 <th>거래량</th>
 </tr>
 <tr>
 <td colspan="7" height="8"></td>
 </tr>
 <tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
 <td align="center"><span class="tah p10 gray03">2021.10.14</span></td>
 <td class="num"><span class="tah p11">39,750</span></td>
 <td class="num">
 <img alt="상승" height="6" src="https://ssl.pstatic.net/imgstock/images/images4/ico_up.gif" style="margin-right:4px;" width="7"/><span class="tah p11 red02">
 				2,400
 				</span>
 </td>
 <td class="num"><span class="tah p11">37,950</span></td>
 <td class="num"><span class="tah p11">40,150</span></td>
 <td class="num"><span class="tah p11">37,900</span></td>
 <td class="num"><span class="tah p11">401,322</span></td>
 </tr>
 <tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
 <td align="center"><span class="tah p10 gray03">2021.10.13</span></td>
 <td class="num"><span class="ta

In [56]:
pd.read_html(str(table))[0]

,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021.10.14,39750.0,2400.0,37950.0,40150.0,37900.0,401322.0
2,2021.10.13,37350.0,0.0,37250.0,37950.0,37100.0,90031.0
3,2021.10.12,37350.0,1100.0,37750.0,37900.0,36850.0,117988.0
4,2021.10.08,38450.0,2150.0,36850.0,38650.0,36600.0,291687.0
5,2021.10.07,36300.0,500.0,36000.0,36700.0,36000.0,141472.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021.10.06,35800.0,800.0,37100.0,37600.0,35750.0,279575.0


In [57]:
temp = pd.read_html(str(table))[0]
temp = temp.dropna()
temp

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.10.14,39750.0,2400.0,37950.0,40150.0,37900.0,401322.0
2,2021.10.13,37350.0,0.0,37250.0,37950.0,37100.0,90031.0
3,2021.10.12,37350.0,1100.0,37750.0,37900.0,36850.0,117988.0
4,2021.10.08,38450.0,2150.0,36850.0,38650.0,36600.0,291687.0
5,2021.10.07,36300.0,500.0,36000.0,36700.0,36000.0,141472.0
9,2021.10.06,35800.0,800.0,37100.0,37600.0,35750.0,279575.0
10,2021.10.05,36600.0,2050.0,38200.0,38350.0,36500.0,385701.0
11,2021.10.01,38650.0,950.0,39150.0,39500.0,38400.0,238094.0
12,2021.09.30,39600.0,200.0,39550.0,40100.0,39250.0,187936.0
13,2021.09.29,39400.0,1300.0,40050.0,40250.0,39250.0,389810.0


In [71]:
# 종목 번호를 이용해 page에 따라 데이터를 읽어오는 함수
# """ 는 이 두개 사이의 행들은 주석 처리되며, 함수의 docstring 으로 사용됩니다.

def get_day_list(item_code, page_no):
    """
    일자별 시세를 페이지별로 수집
    """
    url = f"https://finance.naver.com/item/sise_day.naver?code={item_code}&page={page_no}"
    
    # requests url
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    # BeautifulSoup으로 감싸주기
    html = bs(response.text)
    table = html.select("table")
    table = pd.read_html(str(table))
    temp = table[0].dropna()
    return temp

In [76]:
# 함수가 잘 만들어졌는지 확인하기
get_day_list(item_code, 4)

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.08.26,51100.0,1100.0,52200.0,52500.0,50500.0,701564.0
2,2021.08.25,52200.0,3000.0,49600.0,52600.0,48650.0,2280134.0
3,2021.08.24,49200.0,1200.0,50400.0,51200.0,48650.0,1640089.0
4,2021.08.23,50400.0,3000.0,54100.0,54300.0,50000.0,1736327.0
5,2021.08.20,53400.0,2100.0,56500.0,57900.0,52700.0,2202369.0
9,2021.08.19,55500.0,2000.0,57500.0,60900.0,54400.0,10462187.0


## 반복문을 통한 전체 일자 데이터 수집하기
* (주의) 기간이 긴 데이터를 수집할때는 서버에 부담을 주지 않기 위해 time.sleep()값을 주세요.

In [89]:
import time
# web page 시작번호
page_no = 1
# 데이터를 저장할 빈 변수 선언
item_list = []

while True:
    print(page_no)
    temp = get_day_list(item_code, page_no)
    item_list.append(temp)
    
    page_no = page_no + 1
    time.sleep(0.3)  # 디도스 공격으로 오인할 수 있기 때문
    
    if temp.shape[0] < 10:
        break;

1
2
3
4


In [92]:
# import time
# # web page 시작번호
# page_no = 1
# # 데이터를 저장할 빈 변수 선언
# item_list = []

# while True:
#     print(page_no)
#     temp = get_day_list(item_code, page_no)
#     item_list.append(temp)
    
#     page_no = page_no + 1
#     time.sleep(0.3)  # 디도스 공격으로 오인할 수 있기 때문
    
#     if page_no > 10:
#         break;

In [90]:
len(item_list)

4

## 수집한 데이터 하나의 데이터프레임으로 합치기

<img src="https://pandas.pydata.org/docs/_images/merging_concat_basic.png">

* [Merge, join, concatenate and compare documentation](https://pandas.pydata.org/docs/user_guide/merging.html#merge-join-concatenate-and-compare)

In [93]:
# DataFrame들이 list의 형태로 저장된 것을 concat을 이용하여 합치면서 하나의 DataFrame으로 만듭니다.
df = pd.concat(item_list)
df

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.10.14,39750.0,2400.0,37950.0,40150.0,37900.0,401322.0
2,2021.10.13,37350.0,0.0,37250.0,37950.0,37100.0,90031.0
3,2021.10.12,37350.0,1100.0,37750.0,37900.0,36850.0,117988.0
4,2021.10.08,38450.0,2150.0,36850.0,38650.0,36600.0,291687.0
5,2021.10.07,36300.0,500.0,36000.0,36700.0,36000.0,141472.0
9,2021.10.06,35800.0,800.0,37100.0,37600.0,35750.0,279575.0
10,2021.10.05,36600.0,2050.0,38200.0,38350.0,36500.0,385701.0
11,2021.10.01,38650.0,950.0,39150.0,39500.0,38400.0,238094.0
12,2021.09.30,39600.0,200.0,39550.0,40100.0,39250.0,187936.0
13,2021.09.29,39400.0,1300.0,40050.0,40250.0,39250.0,389810.0


<img src="https://pandas.pydata.org/docs/_images/08_concat_row.svg">

In [94]:
# head와 tail로 데이터의 일부를 가져와서 봅니다.
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.10.14,39750.0,2400.0,37950.0,40150.0,37900.0,401322.0
2,2021.10.13,37350.0,0.0,37250.0,37950.0,37100.0,90031.0
3,2021.10.12,37350.0,1100.0,37750.0,37900.0,36850.0,117988.0
4,2021.10.08,38450.0,2150.0,36850.0,38650.0,36600.0,291687.0
5,2021.10.07,36300.0,500.0,36000.0,36700.0,36000.0,141472.0


In [95]:
df.tail()

,날짜,종가,전일비,시가,고가,저가,거래량
9,2021.10.06,35800.0,800.0,37100.0,37600.0,35750.0,279575.0
10,2021.10.05,36600.0,2050.0,38200.0,38350.0,36500.0,385701.0
11,2021.10.01,38650.0,950.0,39150.0,39500.0,38400.0,238094.0
12,2021.09.30,39600.0,200.0,39550.0,40100.0,39250.0,187936.0
13,2021.09.29,39400.0,1300.0,40050.0,40250.0,39250.0,389810.0


## 데이터프레임에 종목코드와 종목명을 추가하기
* 파생변수 만들기

In [96]:
# '종목코드'와 '종목명' column을 추가하면서 각각 item_code와 item_name 값을 입력합니다.
df["종목코드"] = item_code
df["종목명"] = item_name

In [97]:
df

,날짜,종가,전일비,시가,고가,저가,거래량,종목코드,종목명
1,2021.10.14,39750.0,2400.0,37950.0,40150.0,37900.0,401322.0,089860,롯데렌탈
2,2021.10.13,37350.0,0.0,37250.0,37950.0,37100.0,90031.0,089860,롯데렌탈
3,2021.10.12,37350.0,1100.0,37750.0,37900.0,36850.0,117988.0,089860,롯데렌탈
4,2021.10.08,38450.0,2150.0,36850.0,38650.0,36600.0,291687.0,089860,롯데렌탈
5,2021.10.07,36300.0,500.0,36000.0,36700.0,36000.0,141472.0,089860,롯데렌탈
9,2021.10.06,35800.0,800.0,37100.0,37600.0,35750.0,279575.0,089860,롯데렌탈
10,2021.10.05,36600.0,2050.0,38200.0,38350.0,36500.0,385701.0,089860,롯데렌탈
11,2021.10.01,38650.0,950.0,39150.0,39500.0,38400.0,238094.0,089860,롯데렌탈
12,2021.09.30,39600.0,200.0,39550.0,40100.0,39250.0,187936.0,089860,롯데렌탈
13,2021.09.29,39400.0,1300.0,40050.0,40250.0,39250.0,389810.0,089860,롯데렌탈


## 컬럼 순서 변경하기

In [98]:
# DataFrame에서 column 들의 이름을 순서를 조정하여 column순서를 변경할 수 있습니다.
df.columns

Index(['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량', '종목코드', '종목명'], dtype='object')

In [100]:
cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
df = df[cols]
df

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
1,089860,롯데렌탈,2021.10.14,39750.0,2400.0,37950.0,40150.0,37900.0,401322.0
2,089860,롯데렌탈,2021.10.13,37350.0,0.0,37250.0,37950.0,37100.0,90031.0
3,089860,롯데렌탈,2021.10.12,37350.0,1100.0,37750.0,37900.0,36850.0,117988.0
4,089860,롯데렌탈,2021.10.08,38450.0,2150.0,36850.0,38650.0,36600.0,291687.0
5,089860,롯데렌탈,2021.10.07,36300.0,500.0,36000.0,36700.0,36000.0,141472.0
9,089860,롯데렌탈,2021.10.06,35800.0,800.0,37100.0,37600.0,35750.0,279575.0
10,089860,롯데렌탈,2021.10.05,36600.0,2050.0,38200.0,38350.0,36500.0,385701.0
11,089860,롯데렌탈,2021.10.01,38650.0,950.0,39150.0,39500.0,38400.0,238094.0
12,089860,롯데렌탈,2021.09.30,39600.0,200.0,39550.0,40100.0,39250.0,187936.0
13,089860,롯데렌탈,2021.09.29,39400.0,1300.0,40050.0,40250.0,39250.0,389810.0


In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 1 to 13
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   종목코드    10 non-null     object 
 1   종목명     10 non-null     object 
 2   날짜      10 non-null     object 
 3   종가      10 non-null     float64
 4   전일비     10 non-null     float64
 5   시가      10 non-null     float64
 6   고가      10 non-null     float64
 7   저가      10 non-null     float64
 8   거래량     10 non-null     float64
dtypes: float64(6), object(3)
memory usage: 800.0+ bytes


## 중복데이터 제거하기
* drop_duplicates 를 통해 중복된 데이터가 있다면 제거합니다.

In [103]:
# drop_duplicates : row들 끼리 data를 비교하여 같은 값이 있으면 row중 하나를 삭제
# df.shape를 전/후로 출력하여 삭제된 row가 있는지 확인
df = df.drop_duplicates()
df.shape

(10, 9)

## 기술통계값 구하기

In [111]:
# describe는 기본적으로 수치데이터에 대한 기술통계값을 구하게 됩니다.
df.describe()

,종가,전일비,시가,고가,저가,거래량
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,37925.000000,1145.000000,37985.000000,38715.000000,37360.000000,252361.600000
std,1440.919845,830.478443,1279.116449,1231.090303,1273.839341,116526.548741
min,35800.000000,0.000000,36000.000000,36700.000000,35750.000000,90031.000000
25%,36787.500000,575.000000,37137.500000,37912.500000,36525.000000,153088.000000
50%,37900.000000,1025.000000,37850.000000,38500.000000,36975.000000,258834.500000
75%,39212.500000,1862.500000,38912.500000,39950.000000,38275.000000,362197.500000
max,39750.000000,2400.000000,40050.000000,40250.000000,39250.000000,401322.000000


* 참고 : E notation
[과학적 기수법 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EA%B3%BC%ED%95%99%EC%A0%81_%EA%B8%B0%EC%88%98%EB%B2%95)

과학적 기수법, 과학적 표기법(scientific notation, scientific form, standard index form, standard form)은 너무 크거나 너무 작은 숫자들을 십진법으로 편하게 작성하여 표현하는 방법이다. 과학자, 수학자, 공학자들이 공통적으로 사용하는데, 부분적인 이유는 특정한 산술을 단순화시켜 주기 때문이다. 과학 계산기에서는 "SCI" 디스플레이 모드라는 이름으로 알려져 있다.

In [ ]:
# 거래량의 과학적 기수법 읽기

## 최근 날짜 구해서 파일명 만들기

In [105]:
# 날짜 column의 첫 row값 확인
date = df.iloc[0]["날짜"]
date

'2021.10.14'

In [106]:
# 종목명, 종목코드, 날짜를 이름으로 하는 csv 파일명 만들기
file_name = f"{item_name}_{item_code}_{date}.csv"
file_name

'롯데렌탈_089860_2021.10.14.csv'

## 파일로 저장하기

In [107]:
# 파일로 저장하기 
# index=False 로 데이터프레임의 기본 index 는 저장하지 않도록 합니다.
df.to_csv(file_name, index=False)

In [108]:
# 제대로 저장되었는지 파일을 읽어서 확인합니다.
pd.read_csv(file_name)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
0,89860,롯데렌탈,2021.10.14,39750.0,2400.0,37950.0,40150.0,37900.0,401322.0
1,89860,롯데렌탈,2021.10.13,37350.0,0.0,37250.0,37950.0,37100.0,90031.0
2,89860,롯데렌탈,2021.10.12,37350.0,1100.0,37750.0,37900.0,36850.0,117988.0
3,89860,롯데렌탈,2021.10.08,38450.0,2150.0,36850.0,38650.0,36600.0,291687.0
4,89860,롯데렌탈,2021.10.07,36300.0,500.0,36000.0,36700.0,36000.0,141472.0
5,89860,롯데렌탈,2021.10.06,35800.0,800.0,37100.0,37600.0,35750.0,279575.0
6,89860,롯데렌탈,2021.10.05,36600.0,2050.0,38200.0,38350.0,36500.0,385701.0
7,89860,롯데렌탈,2021.10.01,38650.0,950.0,39150.0,39500.0,38400.0,238094.0
8,89860,롯데렌탈,2021.09.30,39600.0,200.0,39550.0,40100.0,39250.0,187936.0
9,89860,롯데렌탈,2021.09.29,39400.0,1300.0,40050.0,40250.0,39250.0,389810.0


## 전체 과정을 하나의 함수로 만들기

In [112]:
def get_item_list(item_code, item_name):
    """
    일별 시세를 수집하는 함수
    """
    # web page 시작번호
    page_no = 1
    # 데이터를 저장할 빈 변수 선언
    item_list = []

    while True:
        temp = get_day_list(item_code, page_no)
        item_list.append(temp)
    
        page_no = page_no + 1
        time.sleep(0.3)  # 디도스 공격으로 오인할 수 있기 때문
    
        if temp.shape[0] < 10:
            break;
            
        df = pd.concat(item_list)
        df["종목코드"] = item_code
        df["종목명"] = item_name
        cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
        df = df[cols]
        
        return df

In [118]:
# item_code = "352820"
# item_name = "하이브"
# 빅히트의 사명이 하이브로 변경되었습니다.

# item_code = "326030"
# item_name = "SK바이오팜"

item_code = "089860"
item_name = "롯데렌탈"

get_item_list(item_code, item_name)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
1,089860,롯데렌탈,2021.10.14,39750.0,2400.0,37950.0,40150.0,37900.0,401322.0
2,089860,롯데렌탈,2021.10.13,37350.0,0.0,37250.0,37950.0,37100.0,90031.0
3,089860,롯데렌탈,2021.10.12,37350.0,1100.0,37750.0,37900.0,36850.0,117988.0
4,089860,롯데렌탈,2021.10.08,38450.0,2150.0,36850.0,38650.0,36600.0,291687.0
5,089860,롯데렌탈,2021.10.07,36300.0,500.0,36000.0,36700.0,36000.0,141472.0
9,089860,롯데렌탈,2021.10.06,35800.0,800.0,37100.0,37600.0,35750.0,279575.0
10,089860,롯데렌탈,2021.10.05,36600.0,2050.0,38200.0,38350.0,36500.0,385701.0
11,089860,롯데렌탈,2021.10.01,38650.0,950.0,39150.0,39500.0,38400.0,238094.0
12,089860,롯데렌탈,2021.09.30,39600.0,200.0,39550.0,40100.0,39250.0,187936.0
13,089860,롯데렌탈,2021.09.29,39400.0,1300.0,40050.0,40250.0,39250.0,389810.0


## 내가 만든 함수의 기능과 소스코드가 궁금하다면?

In [ ]:
# 도움말 보기
# get_item_list?


In [ ]:
# 소스코드 보기
# get_item_list??
